In [1]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame

In [2]:
connection = create_connection("../database/crypto_billionairs.db")

In [3]:
table = "ETHUSDT_5m_complete_history_long"

In [4]:
df_long = pd.read_sql_query(f'select * from {table}', connection)

In [5]:
df_backtesting = pd.DataFrame()
if "5m" not in table:
    df_backtesting["time"] = pd.to_datetime(df_long["open time"], unit='ms')
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]
    df_backtesting = df_backtesting.set_index("time")

else:
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]

In [6]:
df_backtesting["time"] = pd.to_datetime(df_long["open time"], unit='ms')
df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]
df_backtesting = df_backtesting.set_index("time")

In [10]:
def calculate_osc(data):
    
   
    df_temp = pd.DataFrame()    
    stock = StockDataFrame.retype(data)
    
    df_temp["stochastic_oscillator"] = stock.get('kdjk')
    
    return df_temp["stochastic_oscillator"]

def calucalte_rsi(data):
    
    df_temp = pd.DataFrame()    
    stock = StockDataFrame.retype(data)
    
    df_temp["relative_strength_index"] = stock.get("rsi_30")
    
    return df_temp["relative_strength_index"]

def momentum(df, lag):
    return df["close"].pct_change(periods=lag)

def init_close_long_signal(df):
    
    return df_long['close_indicator']

def init_buy_signal(df):
    
    return df_long['buy_indicator']

    

In [11]:
class OscRsi_5m_long(Strategy):
   
    lag = 4
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.osc = self.I(calculate_osc, self.data.df)
        self.rsi = self.I(calucalte_rsi, self.data.df)
        self.buy_init = self.I(init_buy_signal, self.data.df)
        self.close_buy_init = self.I(init_close_long_signal, self.data.df)
        self.data.df[['Open', 'High', 'Low', 'Close', "Volume",]] = self.data.df[['open', 'high', 'low', 'close', "volume"]]
        self.returns = self.I(momentum, self.data.df, self.lag)
        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume", "kdjk", "rsi_30"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.buy_init[-1] == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        elif self.close_buy_init[-1] == 1 and self.position.is_long:
             self.position.close()
     
        

       

In [12]:
bt = Backtest(df_backtesting, OscRsi_5m_long, cash=10_000, commission=.001)
stats = bt.run()
stats

Start                     2021-03-01 00:00:00
End                       2022-03-31 23:55:00
Duration                    395 days 23:55:00
Exposure Time [%]                     60.9172
Equity Final [$]                      59578.9
Equity Peak [$]                       63071.1
Return [%]                            495.789
Buy & Hold Return [%]                 131.947
Return (Ann.) [%]                     383.618
Volatility (Ann.) [%]                 426.729
Sharpe Ratio                         0.898973
Sortino Ratio                          7.8908
Calmar Ratio                          6.52622
Max. Drawdown [%]                     -58.781
Avg. Drawdown [%]                    -1.56733
Max. Drawdown Duration       85 days 08:15:00
Avg. Drawdown Duration        0 days 22:21:00
# Trades                                   11
Win Rate [%]                              100
Best Trade [%]                        80.4702
Worst Trade [%]                       1.67896
Avg. Trade [%]                    

In [13]:
bt.plot()

C:\Users\janfa\anaconda3\lib\site-packages\backtesting\_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '1H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


ValueError: Length of passed values is 2, index implies 1.

In [49]:
# %%time 
# stats = bt.optimize(lag=[12, 24, 48, 96, 120, 144, 168],
#                     sell_ = np.arange(60, 90, 3).tolist(),#sell_threshold = np.arange(0, -0.15, -0.01).tolist(),
#                     rsi_lower_bound = np.arange(10, 40, 3).tolist(),
#                     rsi_upper_bound = np.arange(12, 50, 3).tolist(),
#                     #osc_lower_bound = np.arange(10, 90, 5).tolist(),
#                     #osc_upper_bound = np.arange(10, 100, 5).tolist(),
#                     #maximize='Equity Final [$]')#,
#                     constraint= lambda param: param.rsi_lower_bound < param.rsi_upper_bound)
# stats

In [50]:
stats._strategy

<Strategy OscRsi_5m_long>

In [51]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,6,6,2018,1440.53910,1735.00,1766.76540,0.204410,6,2018,2012
1,6,3583,5599,1875.81394,1843.26,-195.32364,-0.017355,3583,5599,2016
2,6,8176,10192,1766.13437,2042.80,1659.99378,0.156650,8176,10192,2016
3,6,11558,13574,2172.75058,2486.14,1880.33652,0.144236,11558,13574,2016
4,4,19660,21676,3673.13947,3901.89,915.00212,0.062277,19660,21676,2016
5,7,22803,24819,2008.68668,2843.39,5842.92324,0.415547,22803,24819,2016
6,9,32122,34138,2263.52126,1820.28,-3989.17134,-0.195819,32122,34138,2016
7,9,34153,36169,1950.84890,2367.13,3746.52990,0.213385,34153,36169,2016
8,11,40857,42873,1862.91105,2298.26,4788.83845,0.233693,40857,42873,2016
9,10,44995,47011,2621.05844,3247.54,6264.81560,0.239019,44995,47011,2016


In [52]:
trades = pd.DataFrame(stats['_trades'])

In [53]:
trades.to_excel(f"trades_{table}.xlsx")